In [44]:
import sys; sys.path.append('../'); sys.path.append('../Preprocess')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


# use sklearn metrics, single function
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import Preprocess.preprocessor as preprocessor
from imblearn.over_sampling import SMOTE

import xgboost as xgb
from lightgbm.sklearn import LGBMClassifier
from catboost import CatBoostClassifier

from mlxtend.evaluate import bias_variance_decomp

In [12]:
df_h = preprocessor.ReadData(pth='body_level_classification_train.csv', label='Body_Level')
df_h.head()

,Gender,Age,Height,Weight,H_Cal_Consump,Veg_Consump,Water_Consump,Alcohol_Consump,Smoking,Meal_Count,Food_Between_Meals,Fam_Hist,H_Cal_Burn,Phys_Act,Time_E_Dev,Transport,Body_Level
0,Female,22.547298,1.722461,51.881263,yes,2.663421,1.041110,no,no,3.000000,Frequently,yes,no,0.794402,1.391948,Public_Transportation,Body Level 1
1,Male,19.799054,1.743702,54.927529,yes,2.000000,2.847264,Sometimes,no,3.289260,Sometimes,yes,no,1.680844,2.000000,Public_Transportation,Body Level 1
2,Female,17.823438,1.708406,50.000000,yes,1.642241,1.099231,Sometimes,no,3.452590,Sometimes,no,no,0.418875,1.000000,Public_Transportation,Body Level 1
3,Female,19.007177,1.690727,49.895716,yes,1.212908,1.029703,Sometimes,no,3.207071,Sometimes,no,no,2.000000,1.000000,Public_Transportation,Body Level 1
4,Male,19.729250,1.793315,58.195150,yes,2.508835,2.076933,no,no,3.435905,Sometimes,yes,no,2.026668,1.443328,Automobile,Body Level 1


In [13]:
# add augmented columns like BMI ...
AGGREGATE = True
DISCRETIZE = False
# one hot encoding for categorical columns
ONE_HOT = False
# resample data
RESAMPLE = False
# resmapling using SMOTE
APPLY_SMOTE = False
# Preprocess
df_h = preprocessor.LabelOrdinalEncode(df_h)
if AGGREGATE:
    df_h = preprocessor.Aggregate(df_h, discretize=DISCRETIZE)
if ONE_HOT:
    df_h = preprocessor.OneHotEncode(df_h, label='Body_Level')
if APPLY_SMOTE:
    df_h = preprocessor.SMOTE(df_h)
elif RESAMPLE:
    df_h = preprocessor.Resample(df_h)
#
df_h.head()

,Gender,Age,Height,Weight,H_Cal_Consump,Veg_Consump,Water_Consump,Alcohol_Consump,Smoking,Meal_Count,Food_Between_Meals,Fam_Hist,H_Cal_Burn,Phys_Act,Time_E_Dev,Transport,BMI,Body_Level
0,0,22.547298,1.722461,51.881263,1,2.663421,1.041110,0,0,3.000000,2,1,0,0.794402,1.391948,2,17.486856,0
1,1,19.799054,1.743702,54.927529,1,2.000000,2.847264,1,0,3.289260,1,1,0,1.680844,2.000000,2,18.065315,0
2,0,17.823438,1.708406,50.000000,1,1.642241,1.099231,1,0,3.452590,1,0,0,0.418875,1.000000,2,17.131202,0
3,0,19.007177,1.690727,49.895716,1,1.212908,1.029703,1,0,3.207071,1,0,0,2.000000,1.000000,2,17.454857,0
4,1,19.729250,1.793315,58.195150,1,2.508835,2.076933,0,0,3.435905,1,1,0,2.026668,1.443328,4,18.095627,0


In [14]:
X_train, X_test, y_train, y_test = preprocessor.Split(df_h, test_size=0.2, random_state=42)

print('X_train.shape: ', X_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

print(f'{X_train.head()=}')
print(f'{y_train.head()=}')

X_train.shape:  (1181, 17)
X_test.shape:  (296, 17)
y_train.shape:  (1181,)
y_test.shape:  (296,)
X_train.head()=      Gender        Age    Height      Weight  H_Cal_Consump  Veg_Consump  \
660        1  33.081600  1.705617   83.016968              1     2.000000   
933        0  26.000000  1.610636  105.423532              1     3.000000   
254        0  18.000000  1.600000   55.000000              1     2.000000   
1117       1  30.607546  1.757132  118.565568              1     2.918113   
812        1  40.501722  1.744974  111.169678              1     2.294259   

      Water_Consump  Alcohol_Consump  Smoking  Meal_Count  Food_Between_Meals  \
660        2.991671                0        0    2.797600                   1   
933        2.180566                1        0    3.000000                   1   
254        2.000000                1        0    4.000000                   2   
1117       2.240463                1        0    3.000000                   1   
812        1.870290

In [44]:
def RunModel(model, X_train, y_train, X_test, y_test, name=None,
             Train_Report=False, Train_Conf_Mat=False, Train_F1=False,
             Test_Report=False, Test_Conf_Mat=False, Test_F1=False,
             Bias_Var=False, Bias_Var_Rounds=50):
    #
    if name is None:
        name = model.__class__.__name__
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred = model.predict(X_test)

    if name == 'Linear Regression':
      y_pred, y_pred_train = np.round(y_pred), np.round(y_pred_train)
      y_pred = np.where(y_pred == 4, 3, y_pred)
      y_pred_train = np.where(y_pred_train == 4, 3, y_pred_train)

    if Train_Report or Train_Conf_Mat or Train_F1:
        print(f'Training Metrics for {name}:')
        if Train_Report:
            print(classification_report(y_train, y_pred_train, digits=4, zero_division=0))
        if Train_Conf_Mat:
            print(confusion_matrix(y_train, y_pred_train))
        if Train_F1:
            print(classification_report(
                y_train, y_pred_train, digits=4, zero_division=0).split('\n')[-2])

    if Test_Report or Test_Conf_Mat or Test_F1:
        print(f'\nTesting Metrics for {name}:')
        if Test_Report:
            print(classification_report(y_test, y_pred, digits=4, zero_division=0))
        if Test_Conf_Mat:
            print(confusion_matrix(y_test, y_pred))
        if Test_F1:
            print(classification_report(
                y_test, y_pred, digits=4, zero_division=0).split('\n')[-2])

    if Bias_Var:
        print(f'\nBias-Variance Decomposition for {name}:')
        avg_expected_loss, avg_bias, avg_var = \
            bias_variance_decomp(model, X_train.values, y_train.values,
                                 X_test.values, y_test.values,
                                 loss='0-1_loss', random_seed=42, num_rounds=Bias_Var_Rounds)

        print(f'Avg. Exp. Loss: {avg_expected_loss:.4f}')
        print(f'Avg. Bias: {avg_bias:.4f}')
        print(f'Avg. Variance: {avg_var:.4f}')

    return model

# XGBOOST

In [16]:
# XGBoost
xgb_model = xgb.XGBClassifier(
                            booster='gbtree',
                            learning_rate=0.3,
                            max_depth=3,
                            objective="multi:softprob",
                            random_state=42,
                            num_class=4,
                            # eval_metric="auc",
                            eval_metric="mlogloss",
                            )

xgb_model = RunModel(xgb_model, X_train, y_train, X_test, y_test, name='XGBoost',
            Train_Conf_Mat=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for XGBoost:
[[159   0   0   0]
 [  0 156   0   0]
 [  0   0 324   0]
 [  0   0   0 542]]

Testing Metrics for XGBoost:
[[ 31   0   0   0]
 [  0  45   0   0]
 [  0   0  82   0]
 [  0   0   0 138]]

Bias-Variance Decomposition for XGBoost:
Avg. Exp. Loss: 0.0047
Avg. Bias: 0.0000
Avg. Variance: 0.0047


# LightGBM

In [17]:
# lightgbm
lgb_model = LGBMClassifier(
    boosting_type='dart',
    class_weight=None,
    # importance_type='split',
    learning_rate=0.1,
    # max_depth=-1,
    # min_child_samples=20,
    # min_child_weight=0.001,
    # min_split_gain=0.0,
    n_estimators=500,
    # num_leaves=31,
    objective='multiclass',
    # reg_alpha=0.0,
    # reg_lambda=0.,
    # verbose=-10
)

lgb_model = RunModel(lgb_model, X_train, y_train, X_test, y_test, name='LightGBM',
            Train_Conf_Mat=True, Test_Report=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for LightGBM:
[[159   0   0   0]
 [  0 156   0   0]
 [  0   0 324   0]
 [  0   0   0 542]]

Testing Metrics for LightGBM:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        31
           1     1.0000    1.0000    1.0000        45
           2     0.9878    0.9878    0.9878        82
           3     0.9928    0.9928    0.9928       138

    accuracy                         0.9932       296
   macro avg     0.9951    0.9951    0.9951       296
weighted avg     0.9932    0.9932    0.9932       296

[[ 31   0   0   0]
 [  0  45   0   0]
 [  0   0  81   1]
 [  0   0   1 137]]

Bias-Variance Decomposition for LightGBM:
Avg. Exp. Loss: 0.0066
Avg. Bias: 0.0000
Avg. Variance: 0.0066


# Looks like SVM Overfits, specially with high C

In [18]:
# SVM
# svm = SVC(kernel='linear', C=1.0, random_state=0)
svm = SVC(kernel='linear', C= 100, random_state=0)

svm = RunModel(svm, X_train, y_train, X_test, y_test, name='SVM',
               Train_Conf_Mat=True, Train_Report=True,
               Test_Report=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for SVM:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       159
           1     0.9936    1.0000    0.9968       156
           2     1.0000    0.9969    0.9985       324
           3     1.0000    1.0000    1.0000       542

    accuracy                         0.9992      1181
   macro avg     0.9984    0.9992    0.9988      1181
weighted avg     0.9992    0.9992    0.9992      1181

[[159   0   0   0]
 [  0 156   0   0]
 [  0   1 323   0]
 [  0   0   0 542]]

Testing Metrics for SVM:
              precision    recall  f1-score   support

           0     0.9118    1.0000    0.9538        31
           1     0.9524    0.8889    0.9195        45
           2     0.9634    0.9634    0.9634        82
           3     0.9928    0.9928    0.9928       138

    accuracy                         0.9696       296
   macro avg     0.9551    0.9613    0.9574       296
weighted avg     0.9700    0.9696    0.9694       296

[

# Linear Regression

In [45]:
# linear regression
lr = LinearRegression()

lr = RunModel(lr, X_train, y_train, X_test, y_test, name='Linear Regression',
               Train_Conf_Mat=True, Train_Report=True,
               Test_Report=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for Linear Regression:
              precision    recall  f1-score   support

           0     0.9298    0.6667    0.7766       159
           1     0.6036    0.8590    0.7090       156
           2     0.7669    0.8735    0.8167       324
           3     0.9874    0.8672    0.9234       542

    accuracy                         0.8408      1181
   macro avg     0.8219    0.8166    0.8064      1181
weighted avg     0.8685    0.8408    0.8460      1181

[[106  53   0   0]
 [  8 134  14   0]
 [  0  35 283   6]
 [  0   0  72 470]]

Testing Metrics for Linear Regression:
              precision    recall  f1-score   support

           0     0.8800    0.7097    0.7857        31
           1     0.6607    0.8222    0.7327        45
           2     0.7071    0.8537    0.7735        82
           3     0.9828    0.8261    0.8976       138

    accuracy                         0.8209       296
   macro avg     0.8076    0.8029    0.7974       296
weighted avg     0.8467    0

# Logistic Regression

In [27]:
# logistic regression, we have 4 classes
lgr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=10000, random_state=0, penalty='l2')

lgr = RunModel(lgr, X_train, y_train, X_test, y_test,
               Train_Conf_Mat=True, Train_Report=True,
               Test_Report=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for LogisticRegression:
              precision    recall  f1-score   support

           0     0.9938    1.0000    0.9969       159
           1     0.9935    0.9872    0.9904       156
           2     0.9969    0.9938    0.9954       324
           3     0.9982    1.0000    0.9991       542

    accuracy                         0.9966      1181
   macro avg     0.9956    0.9953    0.9954      1181
weighted avg     0.9966    0.9966    0.9966      1181

[[159   0   0   0]
 [  1 154   1   0]
 [  0   1 322   1]
 [  0   0   0 542]]

Testing Metrics for LogisticRegression:
              precision    recall  f1-score   support

           0     0.9394    1.0000    0.9688        31
           1     1.0000    0.9111    0.9535        45
           2     0.9759    0.9878    0.9818        82
           3     0.9928    1.0000    0.9964       138

    accuracy                         0.9831       296
   macro avg     0.9770    0.9747    0.9751       296
weighted avg     0.9836   

# SVM with RBF Kernel

In [30]:
# SVM rbf
svm = SVC(kernel='rbf', C=1.0, random_state=0)
svm = RunModel(svm, X_train, y_train, X_test, y_test, name='SVM with RBF Kernel',
               Train_Conf_Mat=True, Train_Report=True,
               Test_Report=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for SVM with RBF Kernel:
              precision    recall  f1-score   support

           0     0.7662    0.9686    0.8556       159
           1     0.8158    0.3974    0.5345       156
           2     0.7173    0.9321    0.8107       324
           3     0.9731    0.8672    0.9171       542

    accuracy                         0.8366      1181
   macro avg     0.8181    0.7913    0.7795      1181
weighted avg     0.8543    0.8366    0.8291      1181

[[154   5   0   0]
 [ 47  62  47   0]
 [  0   9 302  13]
 [  0   0  72 470]]

Testing Metrics for SVM with RBF Kernel:
              precision    recall  f1-score   support

           0     0.7317    0.9677    0.8333        31
           1     0.8750    0.4667    0.6087        45
           2     0.6842    0.9512    0.7959        82
           3     0.9829    0.8333    0.9020       138

    accuracy                         0.8243       296
   macro avg     0.8185    0.8047    0.7850       296
weighted avg     0.8574 

# Naive Bayes

In [29]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
nb = MultinomialNB()
# nb = GaussianNB( )
# nb = BernoulliNB()
nb = RunModel(nb, X_train, y_train, X_test, y_test,
               Train_Conf_Mat=True, Train_Report=True,
               Test_Report=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for MultinomialNB:
              precision    recall  f1-score   support

           0     0.7598    0.8553    0.8047       159
           1     0.6290    0.2500    0.3578       156
           2     0.5213    0.5278    0.5245       324
           3     0.7680    0.8672    0.8146       542

    accuracy                         0.6909      1181
   macro avg     0.6695    0.6251    0.6254      1181
weighted avg     0.6809    0.6909    0.6733      1181

[[136  11  12   0]
 [ 35  39  73   9]
 [  8  12 171 133]
 [  0   0  72 470]]

Testing Metrics for MultinomialNB:
              precision    recall  f1-score   support

           0     0.7353    0.8065    0.7692        31
           1     0.5455    0.1333    0.2143        45
           2     0.4615    0.5854    0.5161        82
           3     0.7619    0.8116    0.7860       138

    accuracy                         0.6453       296
   macro avg     0.6260    0.5842    0.5714       296
weighted avg     0.6430    0.6453   

# Catboost 

In [32]:
# CatBoost
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_seed=42,
    verbose=False
)

# Fit the model on the training data
model = RunModel(model, X_train, y_train, X_test, y_test,
               Train_Conf_Mat=True, Train_Report=True,
               Test_Report=True, Test_Conf_Mat=True, Bias_Var=False)

Training Metrics for CatBoostClassifier:
              precision    recall  f1-score   support

           0     1.0000    0.9937    0.9968       159
           1     0.9811    1.0000    0.9905       156
           2     1.0000    0.9938    0.9969       324
           3     1.0000    1.0000    1.0000       542

    accuracy                         0.9975      1181
   macro avg     0.9953    0.9969    0.9961      1181
weighted avg     0.9975    0.9975    0.9975      1181

[[158   1   0   0]
 [  0 156   0   0]
 [  0   2 322   0]
 [  0   0   0 542]]

Testing Metrics for CatBoostClassifier:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        31
           1     1.0000    1.0000    1.0000        45
           2     1.0000    1.0000    1.0000        82
           3     1.0000    1.0000    1.0000       138

    accuracy                         1.0000       296
   macro avg     1.0000    1.0000    1.0000       296
weighted avg     1.0000   

# AdaBoost

In [34]:
# AdaBoost
# Create a decision tree as the base estimator:
base_estimator = DecisionTreeClassifier(max_depth=1)

# Create an AdaBoost classifier with 50 estimators:
adaboost = AdaBoostClassifier(estimator=base_estimator, n_estimators=50, random_state=42)

# Train the AdaBoost classifier on the training data:
adaboost = RunModel(adaboost, X_train, y_train, X_test, y_test,
               Train_Conf_Mat=True, Train_Report=True,
               Test_Report=True, Test_Conf_Mat=True, Bias_Var=True)

Training Metrics for AdaBoostClassifier:
              precision    recall  f1-score   support

           0     1.0000    0.9748    0.9873       159
           1     0.9341    1.0000    0.9659       156
           2     0.9937    0.9722    0.9828       324
           3     0.9963    0.9963    0.9963       542

    accuracy                         0.9873      1181
   macro avg     0.9810    0.9858    0.9831      1181
weighted avg     0.9879    0.9873    0.9874      1181

[[155   4   0   0]
 [  0 156   0   0]
 [  0   7 315   2]
 [  0   0   2 540]]

Testing Metrics for AdaBoostClassifier:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000        31
           1     1.0000    1.0000    1.0000        45
           2     1.0000    0.9878    0.9939        82
           3     0.9928    1.0000    0.9964       138

    accuracy                         0.9966       296
   macro avg     0.9982    0.9970    0.9976       296
weighted avg     0.9966   